In [1]:
import numpy as np
import pandas as pd

pd.options.plotting.backend = "plotly"

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_ipcw
from sksurv.util import Surv

maf_df = pd.read_csv("../../data/molecular_train.csv")
maf_eval = pd.read_csv("../../data/molecular_val.csv")


In [2]:
maf_df["CHR"] = maf_df["CHR"].fillna("UNK") 
maf_eval["CHR"] = maf_eval["CHR"].fillna("UNK")

In [3]:
import pandas as pd
import torch
import torch.nn as nn

chr_vals = sorted(maf_df["CHR"].dropna().unique())
chr2idx = {chr_val: i for i, chr_val in enumerate(chr_vals)}

maf_df["CHR_idx"] = maf_df["CHR"].map(chr2idx)
maf_eval["CHR_idx"] = maf_eval["CHR"].map(chr2idx)

embedding_dim = 8  # à ajuster
chr_embedding = nn.Embedding(num_embeddings=len(chr2idx),
                             embedding_dim=embedding_dim)

# Traitement pour maf_df
chr_idx_tensor = torch.tensor(maf_df["CHR_idx"].values, dtype=torch.long)

with torch.no_grad(): 
    chr_emb_tensor = chr_embedding(chr_idx_tensor) 

# Chaque cellule contiendra un vecteur numpy de taille embedding_dim
maf_df["CHR_emb"] = chr_emb_tensor.numpy().tolist()

# Traitement pour maf_eval
chr_idx_tensor_eval = torch.tensor(maf_eval["CHR_idx"].values, dtype=torch.long)

with torch.no_grad(): 
    chr_emb_tensor_eval = chr_embedding(chr_idx_tensor_eval) 

maf_eval["CHR_emb"] = chr_emb_tensor_eval.numpy().tolist()

In [26]:
maf_df.isna().plot(kind="imshow")

In [27]:
maf_eval.isna().plot(kind="imshow")

In [15]:
# Analyse de la variable GENE
print("=== ANALYSE DE LA VARIABLE GENE ===")
print(f"Nombre total de lignes: {len(maf_df)}")
print(f"Nombre de valeurs uniques dans GENE: {maf_df['GENE'].nunique()}")
print(f"Nombre de valeurs manquantes: {maf_df['GENE'].isna().sum()}")
print(f"Pourcentage de valeurs manquantes: {maf_df['GENE'].isna().sum() / len(maf_df) * 100:.2f}%")

print("\n=== TOP 20 DES GENES LES PLUS FREQUENTS ===")
gene_counts = maf_df['GENE'].value_counts()
print(gene_counts.head(20))

print("\n=== STATISTIQUES DESCRIPTIVES ===")
print(f"Nombre moyen de mutations par gène: {len(maf_df) / maf_df['GENE'].nunique():.2f}")
print(f"Médiane des occurrences par gène: {gene_counts.median():.2f}")
print(f"Gène le plus fréquent: {gene_counts.index[0]} ({gene_counts.iloc[0]} occurrences)")

print("\n=== DISTRIBUTION DES FREQUENCES ===")
freq_distribution = gene_counts.value_counts().sort_index()
print("Nombre de gènes avec:")
for freq, count in freq_distribution.head(10).items():
    print(f"  {freq} mutation(s): {count} gènes")

=== ANALYSE DE LA VARIABLE GENE ===
Nombre total de lignes: 10935
Nombre de valeurs uniques dans GENE: 124
Nombre de valeurs manquantes: 0
Pourcentage de valeurs manquantes: 0.00%

=== TOP 20 DES GENES LES PLUS FREQUENTS ===
GENE
TET2      1663
ASXL1      951
SF3B1      775
DNMT3A     604
RUNX1      578
SRSF2      577
TP53       487
STAG2      376
U2AF1      288
EZH2       252
CBL        228
BCOR       213
NRAS       200
ZRSR2      196
DDX41      185
IDH2       166
CUX1       160
NF1        159
PHF6       149
KRAS       133
Name: count, dtype: int64

=== STATISTIQUES DESCRIPTIVES ===
Nombre moyen de mutations par gène: 88.19
Médiane des occurrences par gène: 22.00
Gène le plus fréquent: TET2 (1663 occurrences)

=== DISTRIBUTION DES FREQUENCES ===
Nombre de gènes avec:
  1 mutation(s): 14 gènes
  2 mutation(s): 5 gènes
  3 mutation(s): 8 gènes
  4 mutation(s): 4 gènes
  5 mutation(s): 1 gènes
  6 mutation(s): 2 gènes
  7 mutation(s): 1 gènes
  8 mutation(s): 4 gènes
  9 mutation(s): 3 g

In [11]:
gene_counts = maf_df['PROTEIN_CHANGE'].value_counts()
gene_counts

PROTEIN_CHANGE
p.?                       529
p.K700E                   426
p.G646fs*12               370
p.P95H                    246
p.P95L                    148
                         ... 
p.H717fs*6                  1
p.E1057*                    1
p.D1314fs*49                1
p.Y64_S65insILDTAGQEEY      1
p.C381F                     1
Name: count, Length: 4686, dtype: int64

In [12]:
maf_df

,ID,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,EFFECT,VAF,DEPTH,CHR_idx,CHR_emb
0,P100000,11,119149248.0,119149248.0,G,A,CBL,p.C419Y,non_synonymous_codon,0.0830,1308.0,2,"[-1.7051371335983276, -1.941335916519165, 1.17..."
1,P100000,5,131822301.0,131822301.0,G,T,IRF1,p.Y164*,stop_gained,0.0220,532.0,17,"[-1.3739770650863647, 0.3757626414299011, 0.64..."
2,P100000,3,77694060.0,77694060.0,G,C,ROBO2,p.?,splice_site_variant,0.4100,876.0,15,"[-0.7721591591835022, -0.3339516222476959, 0.3..."
3,P100000,4,106164917.0,106164917.0,G,T,TET2,p.R1262L,non_synonymous_codon,0.4300,826.0,16,"[-0.11987464874982834, -0.052592065185308456, ..."
4,P100000,2,25468147.0,25468163.0,ACGAAGAGGGGGTGTTC,A,DNMT3A,p.E505fs*141,frameshift_variant,0.0898,942.0,11,"[0.12719504535198212, 0.6901341676712036, -0.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10930,P131472,UNK,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,NaN,NaN,22,"[-0.11210711300373077, 0.8132025599479675, 0.2..."
10931,P131505,UNK,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,NaN,NaN,22,"[-0.11210711300373077, 0.8132025599479675, 0.2..."
10932,P131816,UNK,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,NaN,NaN,22,"[-0.11210711300373077, 0.8132025599479675, 0.2..."
10933,P132717,UNK,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,NaN,NaN,22,"[-0.11210711300373077, 0.8132025599479675, 0.2..."


In [ ]:
import torch
import esm

# 1) Charger le modèle et le tokenizer
model_name = "esm2_t30_150M_UR50D"  # tu peux changer pour un plus gros
model, alphabet = esm.pretrained.load_model_and_alphabet(model_name)
model.eval()  # mode évaluation

# 2) Batch converter pour transformer les séquences en tenseurs
batch_converter = alphabet.get_batch_converter()

# 3) Exemple de séquences (id, description, sequence)
data = [
    ("seq1", "my protein", "MKTFFVLLLCTFTVVSA"),
    ("seq2", "another",    "GAVLIPFWY")
]

batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Optionnel : envoyer sur GPU si dispo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
batch_tokens = batch_tokens.to(device)

# 4) Passer dans le modèle
with torch.no_grad():
    out = model(batch_tokens, repr_layers=[model.num_layers], return_contacts=False)

token_representations = out["representations"][model.num_layers]  # (batch, seq_len, dim)


In [21]:
# Analyse et échantillonnage de PROTEIN_CHANGE
print("=== ÉCHANTILLON DE VALEURS PROTEIN_CHANGE ===")

# Récupérer les valeurs non-nulles et uniques
protein_changes = maf_df['PROTEIN_CHANGE'].dropna().unique()

print(f"Nombre total de valeurs uniques: {len(protein_changes)}")
print(f"Nombre de valeurs manquantes: {maf_df['PROTEIN_CHANGE'].isna().sum()}")

# Prendre un échantillon de 30 valeurs
import random
random.seed(42)  # Pour la reproductibilité

# Si moins de 30 valeurs, prendre toutes
if len(protein_changes) <= 30:
    sample_changes = protein_changes.tolist()
else:
    sample_changes = random.sample(protein_changes.tolist(), 30)

print(f"\n=== ÉCHANTILLON DE 30 VALEURS ===")
for i, change in enumerate(sample_changes, 1):
    print(f"{i:2d}. {change}")

print(f"\n=== TYPES DE CHANGEMENTS OBSERVÉS ===")
# Analyser les patterns
synonymous = [c for c in sample_changes if 'p.' in str(c) and '=' in str(c)]
missense = [c for c in sample_changes if 'p.' in str(c) and '>' in str(c)]
nonsense = [c for c in sample_changes if 'p.' in str(c) and '*' in str(c)]
frameshift = [c for c in sample_changes if 'fs' in str(c).lower()]

print(f"Exemples de mutations synonymes (=): {len(synonymous)}")
print(f"Exemples de mutations faux-sens (>): {len(missense)}")
print(f"Exemples de mutations non-sens (*): {len(nonsense)}")
print(f"Exemples de décalage du cadre (fs): {len(frameshift)}")

=== ÉCHANTILLON DE VALEURS PROTEIN_CHANGE ===
Nombre total de valeurs uniques: 4686
Nombre de valeurs manquantes: 12

=== ÉCHANTILLON DE 30 VALEURS ===
 1. p.H1912fs*38
 2. p.R132C
 3. p.R337C
 4. p.S770*
 5. p.D594N
 6. p.G165C
 7. p.C125W
 8. p.N1823fs*11
 9. p.A17E
10. p.L1622fs*1
11. p.R326C
12. p.M504V
13. p.F71L
14. p.P378Q
15. p.R314C
16. p.Q186E
17. p.E1709*
18. p.F1419fs*40
19. p.N1845fs*122
20. p.L614_K618delins*
21. p.F78fs*15
22. p.Y701*
23. p.R198fs*8
24. p.Q1546*
25. p.K774fs*44
26. p.P204S
27. p.Y1579fs*17
28. p.S770W
29. p.R564L
30. p.L98_C99insNFL

=== TYPES DE CHANGEMENTS OBSERVÉS ===
Exemples de mutations synonymes (=): 0
Exemples de mutations faux-sens (>): 0
Exemples de mutations non-sens (*): 14
Exemples de décalage du cadre (fs): 9


In [22]:
# Analyse et échantillonnage de GENE
print("=== ÉCHANTILLON DE VALEURS GENE ===")

# Récupérer les valeurs non-nulles et uniques
genes = maf_df['GENE'].dropna().unique()

print(f"Nombre total de valeurs uniques: {len(genes)}")
print(f"Nombre de valeurs manquantes: {maf_df['GENE'].isna().sum()}")

# Prendre un échantillon de 30 valeurs
import random
random.seed(42)  # Pour la reproductibilité

# Si moins de 30 valeurs, prendre toutes
if len(genes) <= 30:
    sample_genes = genes.tolist()
else:
    sample_genes = random.sample(genes.tolist(), 30)

print(f"\n=== ÉCHANTILLON DE 30 VALEURS ===")
for i, gene in enumerate(sample_genes, 1):
    print(f"{i:2d}. {gene}")

print(f"\n=== TYPES DE GÈNES OBSERVÉS ===")
# Analyser les patterns de noms de gènes
short_names = [g for g in sample_genes if len(str(g)) <= 5]
medium_names = [g for g in sample_genes if 5 < len(str(g)) <= 10]
long_names = [g for g in sample_genes if len(str(g)) > 10]
with_numbers = [g for g in sample_genes if any(char.isdigit() for char in str(g))]

print(f"Gènes avec noms courts (≤5 caractères): {len(short_names)}")
print(f"Gènes avec noms moyens (6-10 caractères): {len(medium_names)}")
print(f"Gènes avec noms longs (>10 caractères): {len(long_names)}")
print(f"Gènes avec des chiffres dans le nom: {len(with_numbers)}")

=== ÉCHANTILLON DE VALEURS GENE ===
Nombre total de valeurs uniques: 124
Nombre de valeurs manquantes: 0

=== ÉCHANTILLON DE 30 VALEURS ===
 1. ARID1A
 2. SF3B1
 3. TET2
 4. CDKN2B
 5. ZRSR2
 6. DDX41
 7. RB1
 8. MYC
 9. CDKN2C
10. EZH2
11. NFE2
12. JAK3
13. SMC1A
14. JAK2
15. EGFR
16. SUZ12
17. DNMT3A
18. BCL10
19. CDK4
20. U2AF2
21. ATRX
22. DHX33
23. BRAF
24. DDX23
25. ETV6
26. BCOR
27. DICER1
28. PRPF8
29. IRF8
30. PRPF40A

=== TYPES DE GÈNES OBSERVÉS ===
Gènes avec noms courts (≤5 caractères): 24
Gènes avec noms moyens (6-10 caractères): 6
Gènes avec noms longs (>10 caractères): 0
Gènes avec des chiffres dans le nom: 25


In [23]:
maf_df.head()

,ID,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,EFFECT,VAF,DEPTH,CHR_idx,CHR_emb
0,P100000,11,119149248.0,119149248.0,G,A,CBL,p.C419Y,non_synonymous_codon,0.0830,1308.0,2,"[-0.2779015302658081, -1.3668746948242188, 0.9..."
1,P100000,5,131822301.0,131822301.0,G,T,IRF1,p.Y164*,stop_gained,0.0220,532.0,17,"[-0.03838486224412918, 0.05611811578273773, 0...."
2,P100000,3,77694060.0,77694060.0,G,C,ROBO2,p.?,splice_site_variant,0.4100,876.0,15,"[-2.1048288345336914, 0.436048686504364, 0.236..."
3,P100000,4,106164917.0,106164917.0,G,T,TET2,p.R1262L,non_synonymous_codon,0.4300,826.0,16,"[0.09518575668334961, 0.35825836658477783, 0.0..."
4,P100000,2,25468147.0,25468163.0,ACGAAGAGGGGGTGTTC,A,DNMT3A,p.E505fs*141,frameshift_variant,0.0898,942.0,11,"[-0.866215705871582, -1.5490642786026, -0.9768..."


In [38]:
import requests
import time

GENES_train = sorted(maf_df['GENE'].dropna().unique().tolist())
GENES_eval = sorted(maf_eval['GENE'].dropna().unique().tolist())
GENES = sorted(set(GENES_train + GENES_eval))

gene_to_protein_seq = {}

for g in GENES:
    query = f"gene_exact:{g}+AND+organism_id:9606+AND+reviewed:true"
    url = (
        "https://rest.uniprot.org/uniprotkb/search"
        f"?query={query}&format=fasta&size=1"
    )
    r = requests.get(url)
    if r.status_code != 200 or not r.text.strip():
        print(f"Pas de résultat clean pour {g}, status={r.status_code}")
        continue

    fasta = r.text.strip().splitlines()
    seq = "".join(line.strip() for line in fasta if not line.startswith(">"))
    gene_to_protein_seq[g] = seq
    print(g, "→ len =", len(seq))

    time.sleep(0.2)  



ABL1 → len = 1130
ARID1A → len = 2285
ARID1A → len = 2285
ARID2 → len = 1835
ARID2 → len = 1835
ASXL1 → len = 1541
ASXL1 → len = 1541
ASXL2 → len = 1435
ASXL2 → len = 1435
ATM → len = 3056
ATM → len = 3056
ATRX → len = 2492
ATRX → len = 2492
BAP1 → len = 1491
BAP1 → len = 1491
BCL10 → len = 233
BCL10 → len = 233
BCL2 → len = 239
BCL2 → len = 239
BCOR → len = 1755
BCOR → len = 1755
BCORL1 → len = 1785
BCORL1 → len = 1785
BLM → len = 1417
BLM → len = 1417
BRAF → len = 766
BRAF → len = 766
BRCA2 → len = 3418
BRCA2 → len = 3418
BRCC3 → len = 316
BRCC3 → len = 316
CALR → len = 417
CALR → len = 417
CBL → len = 906
CBL → len = 906
CCND3 → len = 292
CCND3 → len = 292
CDK4 → len = 303
CDK4 → len = 303
CDKN1B → len = 198
CDKN1B → len = 198
CDKN2A → len = 156
CDKN2A → len = 156
CDKN2B → len = 138
CDKN2B → len = 138
CDKN2C → len = 168
CDKN2C → len = 168
CEBPA → len = 358
CEBPA → len = 358
CHEK2 → len = 543
CHEK2 → len = 543
CREBBP → len = 2442
CREBBP → len = 2442
CSF1R → len = 972
CSF1R → len = 97

In [41]:
gene_to_protein_seq

{'ABL1': 'MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPISPAPVPSTLPSASS

In [44]:

maf_df["protein_seq"] = maf_df["GENE"].map(gene_to_protein_seq)
maf_eval["protein_seq"] = maf_eval["GENE"].map(gene_to_protein_seq)


print(f"maf_df: {maf_df['protein_seq'].notna().sum()} séquences sur {len(maf_df)} lignes")
print(f"maf_eval: {maf_eval['protein_seq'].notna().sum()} séquences sur {len(maf_eval)} lignes")

maf_df: 10935 séquences sur 10935 lignes
maf_eval: 3089 séquences sur 3089 lignes


In [53]:
maf_df["protein_seq"][0]

'MAGNVKKSSGAGGGSGSGGSGSGGLIGLMKDAFQPHHHHHHHLSPHPPGTVDKKMVEKCWKLMDKVVRLCQNPKLALKNSPPYILDLLPDTYQHLRTILSRYEGKMETLGENEYFRVFMENLMKKTKQTISLFKEGKERMYEENSQPRRNLTKLSLIFSHMLAELKGIFPSGLFQGDTFRITKADAAEFWRKAFGEKTIVPWKSFRQALHEVHPISSGLEAMALKSTIDLTCNDYISVFEFDIFTRLFQPWSSLLRNWNSLAVTHPGYMAFLTYDEVKARLQKFIHKPGSYIFRLSCTRLGQWAIGYVTADGNILQTIPHNKPLFQALIDGFREGFYLFPDGRNQNPDLTGLCEPTPQDHIKVTQEQYELYCEMGSTFQLCKICAENDKDVKIEPCGHLMCTSCLTSWQESEGQGCPFCRCEIKGTEPIVVDPFDPRGSGSLLRQGAEGAPSPNYDDDDDERADDTLFMMKELAGAKVERPPSPFSMAPQASLPPVPPRLDLLPQRVCVPSSASALGTASKAASGSLHKDKPLPVPPTLRDLPPPPPPDRPYSVGAESRPQRRPLPCTPGDCPSRDKLPPVPSSRLGDSWLPRPIPKVPVSAPSSSDPWTGRELTNRHSLPFSLPSQMEPRPDVPRLGSTFSLDTSMSMNSSPLVGPECDHPKIKPSSSANAIYSLAARPLPVPKLPPGEQCEGEEDTEYMTPSSRPLRPLDTSQSSRACDCDQQIDSCTYEAMYNIQSQAPSITESSTFGEGNLAAAHANTGPEESENEDDGYDVPKPPVPAVLARRTLSDISNASSSFGWLSLDGDPTTNVTEGSQVPERPPKPFPRRINSERKAGSCQQGSGPAASAATASPQLSSEIENLMSQGYSYQDIQKALVIAQNNIEMAKNILREFVSISSPAHVAT'

In [55]:
import re

# --- Regex HGVS ---------------------------------------------------------

missense_or_nonsense_re = re.compile(r"^p\.([A-Z])(\d+)([A-Z\*])$")

# frameshift : on accepte n'importe quoi après "fs" (fs*38, fs*>5, fsX, etc.)
frameshift_re = re.compile(r"^p\.([A-Z])(\d+)fs.*$")

insertion_re = re.compile(r"^p\.([A-Z])(\d+)_([A-Z])(\d+)ins([A-Z]+)$")

# delins classique : p.L614_K618delins* ou p.L614_K618delinsABC
delins_re = re.compile(r"^p\.([A-Z])(\d+)_([A-Z])(\d+)delins(\*|[A-Z]+)$")

# cas comme p.A78_R83delAVLDGR -> en réalité une delins
del_with_seq_re = re.compile(r"^p\.([A-Z])(\d+)_([A-Z])(\d+)del([A-Z]+)$")

simple_stop_re = re.compile(r"^p\.([A-Z])(\d+)\*$")
simple_del_range_re = re.compile(r"^p\.([A-Z])(\d+)_([A-Z])(\d+)del$")
simple_del_single_re = re.compile(r"^p\.([A-Z])(\d+)del$")




def apply_protein_change(protein_seq: str, protein_change: str) -> str:
    """
    Applique une mutation HGVS protéique sur une séquence d'acides aminés.
    Retourne la séquence mutée.
    En cas de problème (position hors séquence, pattern non géré, etc.),
    la séquence d'origine est renvoyée et un warning est affiché.
    """

    if not isinstance(protein_change, str) or not protein_change.startswith("p."):
        print(f"Mutation HGVS non gérée ou mal parsée : {protein_change}")
        return protein_seq

    # ----------------------------
    # 1. Missense / nonsense simple : p.R132C ou p.S770*
    # ----------------------------
    m_missense = re.match(r"^p\.([A-Z])(\d+)([A-Z*])$", protein_change)
    if m_missense:
        aa_ref, pos_str, aa_alt = m_missense.groups()
        pos = int(pos_str)

        # Position hors séquence -> warning + on renvoie WT
        if pos < 1 or pos > len(protein_seq):
            print(
                f"Warning: position {pos} hors séquence (longueur {len(protein_seq)}) "
                f"pour mutation {protein_change}"
            )
            return protein_seq

        # Vérif acide aminé de référence
        if protein_seq[pos - 1] != aa_ref:
            print(
                f"Warning: HGVS attend {aa_ref}{pos}, "
                f"mais la séquence contient {protein_seq[pos - 1]}{pos}"
            )
            # On continue quand même et on applique la mutation

        # Nonsense : on tronque à la position précédente
        if aa_alt == "*":
            # La position pos devient un STOP -> on garde jusqu'à pos-1
            return protein_seq[: pos - 1]

        # Missense classique
        return protein_seq[: pos - 1] + aa_alt + protein_seq[pos:]

    # ----------------------------
    # 2. Frameshift : p.H1912fs*38 ou p.S810fs*?
    # ----------------------------
    m_fs = re.match(r"^p\.([A-Z])(\d+)fs\*(\d+|\?)$", protein_change)
    if m_fs:
        aa_ref, pos_str, stop_len = m_fs.groups()
        pos = int(pos_str)

        if pos < 1 or pos > len(protein_seq):
            print(
                f"Warning: position {pos} hors séquence (longueur {len(protein_seq)}) "
                f"pour frameshift {protein_change}"
            )
            return protein_seq

        if protein_seq[pos - 1] != aa_ref:
            print(
                f"Warning: HGVS (frameshift) attend {aa_ref}{pos}, "
                f"mais la séquence contient {protein_seq[pos - 1]}{pos}"
            )
            # On continue quand même

        # Sans séquence nucléotidique, on ne peut pas recalculer le nouveau cadre.
        # On approxime en tronquant à la position précédant le codon altéré.
        return protein_seq[: pos - 1]

    # ----------------------------
    # 3. Insertion : p.L98_C99insNFL
    # ----------------------------
    m_ins = re.match(r"^p\.([A-Z])(\d+)_([A-Z])(\d+)ins([A-Z]+)$", protein_change)
    if m_ins:
        aa1, pos1_str, aa2, pos2_str, inserted = m_ins.groups()
        pos1 = int(pos1_str)
        pos2 = int(pos2_str)

        # Vérif grossière des bornes
        if pos1 < 1 or pos1 > len(protein_seq) or pos2 < 1 or pos2 > len(protein_seq):
            print(
                f"Warning: insertion {protein_change} hors séquence "
                f"(longueur {len(protein_seq)})"
            )
            return protein_seq

        # Vérif des AAs de référence
        if protein_seq[pos1 - 1] != aa1 or protein_seq[pos2 - 1] != aa2:
            print(
                f"Warning: HGVS (insertion) attend {aa1}{pos1}_{aa2}{pos2}, "
                f"mais la séquence contient {protein_seq[pos1 - 1]}{pos1}_"
                f"{protein_seq[pos2 - 1]}{pos2}"
            )
            # On continue quand même

        # L'insert est entre pos1 et pos2, on garde pos1, on insère, puis on reprend à pos2
        prefix = protein_seq[:pos1]       # jusqu'à et incluant aa pos1
        suffix = protein_seq[pos1:]       # à partir de aa pos2 (si pos2 = pos1+1)
        return prefix + inserted + suffix

    # ----------------------------
    # 4. delins avec STOP : p.L614_K618delins*
    # ----------------------------
    m_delins_stop = re.match(r"^p\.([A-Z])(\d+)_([A-Z])(\d+)delins\*$", protein_change)
    if m_delins_stop:
        aa1, pos1_str, aa2, pos2_str = m_delins_stop.groups()
        pos1 = int(pos1_str)
        pos2 = int(pos2_str)

        if pos1 < 1 or pos2 > len(protein_seq) or pos1 > pos2:
            print(
                f"Warning: delins* {protein_change} hors séquence "
                f"(longueur {len(protein_seq)})"
            )
            return protein_seq

        if protein_seq[pos1 - 1] != aa1 or protein_seq[pos2 - 1] != aa2:
            print(
                f"Warning: HGVS (delins*) attend {aa1}{pos1}_{aa2}{pos2}, "
                f"mais la séquence contient {protein_seq[pos1 - 1]}{pos1}_"
                f"{protein_seq[pos2 - 1]}{pos2}"
            )
            # On continue quand même

        # La région pos1–pos2 est remplacée par un STOP -> on tronque avant pos1
        return protein_seq[: pos1 - 1]

    # ----------------------------
    # 5. Deletion simple : p.A78_R83del
    # ----------------------------
    m_del = re.match(r"^p\.([A-Z])(\d+)_([A-Z])(\d+)del$", protein_change)
    if m_del:
        aa1, pos1_str, aa2, pos2_str = m_del.groups()
        pos1 = int(pos1_str)
        pos2 = int(pos2_str)

        if pos1 < 1 or pos2 > len(protein_seq) or pos1 > pos2:
            print(
                f"Warning: deletion {protein_change} hors séquence "
                f"(longueur {len(protein_seq)})"
            )
            return protein_seq

        if protein_seq[pos1 - 1] != aa1 or protein_seq[pos2 - 1] != aa2:
            print(
                f"Warning: HGVS (deletion) attend {aa1}{pos1}_{aa2}{pos2}, "
                f"mais la séquence contient {protein_seq[pos1 - 1]}{pos1}_"
                f"{protein_seq[pos2 - 1]}{pos2}"
            )
            # On continue quand même

        # On enlève les aa de pos1 à pos2 inclus
        return protein_seq[: pos1 - 1] + protein_seq[pos2:]

    # ----------------------------
    # 6. Deletion avec séquence explicite : p.A78_R83delAVLDGR
    # ----------------------------
    m_del_with_seq = re.match(r"^p\.([A-Z])(\d+)_([A-Z])(\d+)del([A-Z]+)$", protein_change)
    if m_del_with_seq:
        aa1, pos1_str, aa2, pos2_str, deleted_seq = m_del_with_seq.groups()
        pos1 = int(pos1_str)
        pos2 = int(pos2_str)

        if pos1 < 1 or pos2 > len(protein_seq) or pos1 > pos2:
            print(
                f"Warning: deletion {protein_change} hors séquence "
                f"(longueur {len(protein_seq)})"
            )
            return protein_seq

        region = protein_seq[pos1 - 1:pos2]
        if region != deleted_seq:
            print(
                f"Warning: HGVS (deletion+seq) attend '{deleted_seq}' entre {pos1}-{pos2}, "
                f"mais la séquence contient '{region}'"
            )
            # On continue quand même

        return protein_seq[: pos1 - 1] + protein_seq[pos2:]

    # ----------------------------
    # 7. Cas non gérés
    # ----------------------------
    print(f"Mutation HGVS non gérée ou mal parsée : {protein_change}")
    return protein_seq


In [56]:
# Ajoute la séquence mutée dans une nouvelle colonne
maf_df["mutated_protein_seq"] = maf_df.apply(mutate_row, axis=1)
maf_eval["mutated_protein_seq"] = maf_eval.apply(mutate_row, axis=1)
print(maf_df[["GENE", "PROTEIN_CHANGE", "mutated_protein_seq"]])

Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.Y631fs*>5
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.N293delN
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.V445delV
Mutation HGVS non gérée ou mal parsée : p.M317delM
Mutation HGVS non gérée ou mal parsée : p.?
Mutation HGVS non gérée ou mal parsée : p.?
Mut

In [59]:
# Analyse des résultats de mutation
print("=== ANALYSE DES RÉSULTATS DE MUTATION ===")

# Statistiques générales
total_mutations = len(maf_df)
mutations_with_sequences = maf_df['protein_seq'].notna().sum()
mutations_applied = maf_df['mutated_protein_seq'].notna().sum()

print(f"Nombre total de mutations: {total_mutations}")
print(f"Mutations avec séquences protéiques: {mutations_with_sequences}")
print(f"Mutations traitées: {mutations_applied}")
print(f"Pourcentage de couverture: {mutations_applied/total_mutations*100:.1f}%")

# Analyse des types de problèmes
print(f"\n=== TYPES DE PROBLÈMES RENCONTRÉS ===")

# Compter les différents types de problèmes par analyse du output précédent
problem_types = {
    "Mutations non gérées (p.?)": 0,
    "Warnings HGVS (séquence différente)": 0, 
    "Positions hors séquence": 0,
    "Mutations NaN": 0,
    "Frameshift non standard": 0,
    "Mutations spéciales (FLT3_ITD, MLL_PTD)": 0
}

# Cette analyse sera plus précise avec les vrais logs, mais on peut estimer
print("Les principaux problèmes identifiés:")
print("- Mutations 'p.?' : format HGVS non reconnu")
print("- Warnings HGVS : acides aminés de référence différents entre HGVS et séquence UniProt")
print("- Positions hors séquence : mutation au-delà de la longueur de la protéine") 
print("- Mutations complexes : FLT3_ITD, MLL_PTD (duplications internes)")
print("- Frameshift avec formats non standard")

# Exemples de mutations réussies vs échouées
print(f"\n=== EXEMPLES DE MUTATIONS ===")

# Mutations réussies (séquence différente de l'originale)
successful_mutations = maf_df[
    (maf_df['protein_seq'].notna()) & 
    (maf_df['mutated_protein_seq'].notna()) &
    (maf_df['protein_seq'] != maf_df['mutated_protein_seq'])
]

print(f"Mutations appliquées avec succès (séquence modifiée): {len(successful_mutations)}")

if len(successful_mutations) > 0:
    print("\nExemples de mutations réussies:")
    for i in range(min(5, len(successful_mutations))):
        row = successful_mutations.iloc[i]
        orig_len = len(row['protein_seq']) if row['protein_seq'] else 0
        mut_len = len(row['mutated_protein_seq']) if row['mutated_protein_seq'] else 0
        print(f"  {row['GENE']} {row['PROTEIN_CHANGE']}: {orig_len} → {mut_len} aa")

# Mutations où la séquence n'a pas changé (probablement des erreurs)
unchanged_mutations = maf_df[
    (maf_df['protein_seq'].notna()) & 
    (maf_df['mutated_protein_seq'].notna()) &
    (maf_df['protein_seq'] == maf_df['mutated_protein_seq'])
]

print(f"\nMutations non appliquées (séquence inchangée): {len(unchanged_mutations)}")

if len(unchanged_mutations) > 0:
    print("Exemples de mutations non appliquées:")
    for i in range(min(5, len(unchanged_mutations))):
        row = unchanged_mutations.iloc[i]
        print(f"  {row['GENE']} {row['PROTEIN_CHANGE']}")

=== ANALYSE DES RÉSULTATS DE MUTATION ===
Nombre total de mutations: 10935
Mutations avec séquences protéiques: 10935
Mutations traitées: 10935
Pourcentage de couverture: 100.0%

=== TYPES DE PROBLÈMES RENCONTRÉS ===
Les principaux problèmes identifiés:
- Mutations 'p.?' : format HGVS non reconnu
- Warnings HGVS : acides aminés de référence différents entre HGVS et séquence UniProt
- Positions hors séquence : mutation au-delà de la longueur de la protéine
- Mutations complexes : FLT3_ITD, MLL_PTD (duplications internes)
- Frameshift avec formats non standard

=== EXEMPLES DE MUTATIONS ===
Mutations appliquées avec succès (séquence modifiée): 10119

Exemples de mutations réussies:
  CBL p.C419Y: 906 → 906 aa
  IRF1 p.Y164*: 325 → 163 aa
  TET2 p.R1262L: 2002 → 2002 aa
  DNMT3A p.E505fs*141: 912 → 504 aa
  CHEK2 p.W454*: 543 → 453 aa

Mutations non appliquées (séquence inchangée): 816
Exemples de mutations non appliquées:
  ROBO2 p.?
  TP53 p.?
  EZH2 p.?
  MPL p.Y631fs*>5
  STAG1 p.S831

In [58]:
maf_df

,ID,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,EFFECT,VAF,DEPTH,CHR_idx,CHR_emb,parsed_mutation,mutation_type,mutated_sequence,protein_seq,mutated_protein_seq
0,P100000,11,119149248.0,119149248.0,G,A,CBL,p.C419Y,non_synonymous_codon,0.0830,1308.0,2,"[-0.2779015302658081, -1.3668746948242188, 0.9...","{'type': 'missense', 'wt': 'C', 'pos': 419, 'm...",missense,MAGNVKKSSGAGGGSGSGGSGSGGLIGLMKDAFQPHHHHHHHLSPH...,MAGNVKKSSGAGGGSGSGGSGSGGLIGLMKDAFQPHHHHHHHLSPH...,MAGNVKKSSGAGGGSGSGGSGSGGLIGLMKDAFQPHHHHHHHLSPH...
1,P100000,5,131822301.0,131822301.0,G,T,IRF1,p.Y164*,stop_gained,0.0220,532.0,17,"[-0.03838486224412918, 0.05611811578273773, 0....","{'type': 'nonsense', 'wt': 'Y', 'pos': 164}",nonsense,MPITRMRMRPWLEMQINSNQIPGLIWINKEEMIFQIPWKHAAKHGW...,MPITRMRMRPWLEMQINSNQIPGLIWINKEEMIFQIPWKHAAKHGW...,MPITRMRMRPWLEMQINSNQIPGLIWINKEEMIFQIPWKHAAKHGW...
2,P100000,3,77694060.0,77694060.0,G,C,ROBO2,p.?,splice_site_variant,0.4100,876.0,15,"[-2.1048288345336914, 0.436048686504364, 0.236...",{'type': 'unknown'},unknown,MSLLMFTQLLLCGFLYVRVDGSRLRQEDFPPRIVEHPSDVIVSKGE...,MSLLMFTQLLLCGFLYVRVDGSRLRQEDFPPRIVEHPSDVIVSKGE...,MSLLMFTQLLLCGFLYVRVDGSRLRQEDFPPRIVEHPSDVIVSKGE...
3,P100000,4,106164917.0,106164917.0,G,T,TET2,p.R1262L,non_synonymous_codon,0.4300,826.0,16,"[0.09518575668334961, 0.35825836658477783, 0.0...","{'type': 'missense', 'wt': 'R', 'pos': 1262, '...",missense,MEQDRTNHVEGNRLSPFLIPSPPICQTEPLATKLQNGSPLPERAHP...,MEQDRTNHVEGNRLSPFLIPSPPICQTEPLATKLQNGSPLPERAHP...,MEQDRTNHVEGNRLSPFLIPSPPICQTEPLATKLQNGSPLPERAHP...
4,P100000,2,25468147.0,25468163.0,ACGAAGAGGGGGTGTTC,A,DNMT3A,p.E505fs*141,frameshift_variant,0.0898,942.0,11,"[-0.866215705871582, -1.5490642786026, -0.9768...","{'type': 'frameshift', 'wt': 'E', 'pos': 505, ...",frameshift,MPAMPSSGPGDTSSSAAEREEDRKDGEEQEEPRGKEERQEPSTTAR...,MPAMPSSGPGDTSSSAAEREEDRKDGEEQEEPRGKEERQEPSTTAR...,MPAMPSSGPGDTSSSAAEREEDRKDGEEQEEPRGKEERQEPSTTAR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10930,P131472,UNK,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,NaN,NaN,22,"[0.5754810571670532, -0.6499818563461304, 1.39...",{'type': 'unknown'},unknown,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...
10931,P131505,UNK,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,NaN,NaN,22,"[0.5754810571670532, -0.6499818563461304, 1.39...",{'type': 'unknown'},unknown,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...
10932,P131816,UNK,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,NaN,NaN,22,"[0.5754810571670532, -0.6499818563461304, 1.39...",{'type': 'unknown'},unknown,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...
10933,P132717,UNK,NaN,NaN,NaN,NaN,MLL,MLL_PTD,PTD,NaN,NaN,22,"[0.5754810571670532, -0.6499818563461304, 1.39...",{'type': 'unknown'},unknown,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...,MAHSCRWRFPARPGTTGGGGGGGRRGLGGAPRQRVPALLLPPGPPV...
